In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [6]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,56,2024-02-23,-3,113.872727,98.249091,0.542109,11.454545,25.300000,0.187091,115.101818,...,10.903636,27.734545,0.219945,119.565455,0,2,4,9,28,2024
1,56,2024-02-23,2,117.600000,97.954545,0.570055,12.685455,24.434545,0.241455,119.561818,...,10.979630,24.409259,0.204704,114.335185,0,2,4,9,24,2024
2,55,2024-02-23,-8,114.777778,102.555556,0.541407,11.838889,19.931481,0.177426,111.883333,...,10.512963,20.496296,0.208852,121.314815,0,2,4,9,30,2024
3,54,2024-02-23,1,118.264151,97.177358,0.569472,11.088679,24.662264,0.220736,121.486792,...,12.530357,21.703571,0.181036,108.837500,0,2,4,9,13,2024
4,56,2024-02-23,2,110.581818,96.178182,0.535491,11.245455,22.254545,0.220855,114.770909,...,11.354545,25.581818,0.214545,118.790909,0,2,4,9,16,2024
5,57,2024-02-23,-2,122.017857,100.771429,0.573571,11.041071,21.851786,0.222518,120.405357,...,12.865455,23.187273,0.223655,117.261818,0,2,4,8,17,2024
6,56,2024-02-23,-3,113.945455,95.885455,0.555109,10.692727,26.005455,0.179164,118.985455,...,12.451852,25.998148,0.198870,110.170370,0,2,4,9,8,2024
7,55,2024-02-23,3,108.648148,97.564815,0.529389,11.842593,22.150000,0.169815,111.037037,...,11.930189,27.469811,0.193264,119.524528,0,2,4,9,5,2024
8,56,2024-02-23,-1,111.818182,100.940000,0.527891,12.523636,22.483636,0.173818,110.472727,...,11.892857,19.608929,0.219786,115.710714,0,2,4,9,27,2024


In [7]:
import joblib

In [8]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [9]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-23,TOR,ATL,3.450000,1.327869,0
1,2024-02-23,PHO,HOU,1.555556,2.500000,1
2,2024-02-23,WAS,OKC,10.000000,1.062500,0
3,2024-02-23,LAC,MEM,1.243902,4.300000,1
4,2024-02-23,MIA,NOP,2.240000,1.675676,0
5,2024-02-23,MIL,MIN,2.500000,1.555556,0
6,2024-02-23,DEN,POR,1.235294,4.300000,1
7,2024-02-23,CHO,GSW,7.000000,1.111111,0
8,2024-02-23,SAS,LAL,4.600000,1.212766,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-23,TOR,ATL,3.450000,1.327869,0
1,2024-02-23,PHO,HOU,1.555556,2.500000,1
2,2024-02-23,WAS,OKC,10.000000,1.062500,0
3,2024-02-23,LAC,MEM,1.243902,4.300000,1
4,2024-02-23,MIA,NOP,2.240000,1.675676,0
5,2024-02-23,MIL,MIN,2.500000,1.555556,0
6,2024-02-23,DEN,POR,1.235294,4.300000,1
7,2024-02-23,CHO,GSW,7.000000,1.111111,0
8,2024-02-23,SAS,LAL,4.600000,1.212766,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-23,TOR,ATL,3.450000,1.327869,0
1,2024-02-23,PHO,HOU,1.555556,2.500000,1
2,2024-02-23,WAS,OKC,10.000000,1.062500,0
3,2024-02-23,LAC,MEM,1.243902,4.300000,1
4,2024-02-23,MIA,NOP,2.240000,1.675676,0
5,2024-02-23,MIL,MIN,2.500000,1.555556,0
6,2024-02-23,DEN,POR,1.235294,4.300000,1
7,2024-02-23,CHO,GSW,7.000000,1.111111,0
8,2024-02-23,SAS,LAL,4.600000,1.212766,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-23,TOR,ATL,3.450000,1.327869,0
1,2024-02-23,PHO,HOU,1.555556,2.500000,1
2,2024-02-23,WAS,OKC,10.000000,1.062500,0
3,2024-02-23,LAC,MEM,1.243902,4.300000,1
4,2024-02-23,MIA,NOP,2.240000,1.675676,0
5,2024-02-23,MIL,MIN,2.500000,1.555556,0
6,2024-02-23,DEN,POR,1.235294,4.300000,1
7,2024-02-23,CHO,GSW,7.000000,1.111111,0
8,2024-02-23,SAS,LAL,4.600000,1.212766,0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-23,TOR,ATL,3.450000,1.327869,0.0
1,2024-02-23,PHO,HOU,1.555556,2.500000,1.0
2,2024-02-23,WAS,OKC,10.000000,1.062500,0.0
3,2024-02-23,LAC,MEM,1.243902,4.300000,1.0
4,2024-02-23,MIA,NOP,2.240000,1.675676,0.0
5,2024-02-23,MIL,MIN,2.500000,1.555556,1.0
6,2024-02-23,DEN,POR,1.235294,4.300000,1.0
7,2024-02-23,CHO,GSW,7.000000,1.111111,0.0
8,2024-02-23,SAS,LAL,4.600000,1.212766,0.0


In [14]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-23,TOR,ATL,3.450000,1.327869,0.0
1,2024-02-23,PHO,HOU,1.555556,2.500000,1.0
2,2024-02-23,WAS,OKC,10.000000,1.062500,0.0
3,2024-02-23,LAC,MEM,1.243902,4.300000,1.0
4,2024-02-23,MIA,NOP,2.240000,1.675676,0.0
5,2024-02-23,MIL,MIN,2.500000,1.555556,1.0
6,2024-02-23,DEN,POR,1.235294,4.300000,1.0
7,2024-02-23,CHO,GSW,7.000000,1.111111,0.0
8,2024-02-23,SAS,LAL,4.600000,1.212766,0.0
